In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import os

# Custom activation function for ANN
def custom_relu(x):
    return tf.maximum(0.0, x)

# Custom metric for evaluation (since we cannot use built-in accuracy)
def custom_accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true, axis=1), tf.argmax(y_pred, axis=1)), tf.float32))

# ANN Classifier Class
class ANNClassifier:
    def __init__(self, input_shape, num_classes, hidden_layers_sizes=(64, 64)):
        self.model = Sequential()
        self.model.add(Flatten(input_shape=input_shape))
        for size in hidden_layers_sizes:
            self.model.add(Dense(size, activation=custom_relu))
        self.model.add(Dense(num_classes, activation='softmax'))

    def compile(self, optimizer):
        self.model.compile(optimizer=optimizer, loss=self.custom_loss)

    def fit(self, X, y, epochs=10, batch_size=32):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

    def predict(self, X):
        return self.model.predict(X)

    # Custom loss function (since we cannot use built-in loss functions)
    def custom_loss(self, y_true, y_pred):
        return -tf.reduce_mean(y_true * tf.math.log(y_pred))

# GMM Classifier Class
class GMMClassifier:
    def __init__(self, n_components):
        self.gmm = GaussianMixture(n_components=n_components)

    def fit(self, X):
        self.gmm.fit(X)

    def predict(self, X):
        return self.gmm.predict(X)

# Ensemble Class
class Ensemble:
    def __init__(self, ann, gmm):
        self.ann = ann
        self.gmm = gmm

    def fit(self, X, y):
        # Fit ANN
        self.ann.fit(X, y)
        # Get ANN predictions to use as features for GMM
        ann_predictions = self.ann.predict(X)
        # Fit GMM with features from ANN
        self.gmm.fit(ann_predictions)

    def predict(self, X):
        # Get ANN predictions to use as features for GMM
        ann_predictions = self.ann.predict(X)
        # Predict with GMM
        return self.gmm.predict(ann_predictions)

# Function to load images and preprocess them
def load_images(image_paths, target_size=(32, 32)):
    images = []
    labels = []
    for image_path in image_paths:
        # Load image
        img = Image.open(image_path)
        img = img.resize(target_size)  # Resize image to target size
        img = np.array(img)  # Convert to numpy array
        if img.shape == target_size:  # Grayscale to RGB
            img = np.stack((img,) * 3, axis=-1)
        images.append(img)
        # Extract label from file path
        label = os.path.basename(os.path.dirname(image_path))
        labels.append(label)
    images = np.array(images, dtype='float32') / 255.0  # Normalize pixel values
    # Convert labels to integer encoding
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)  # Convert labels to one-hot encoding
    return images, labels

# Load images
image_paths = glob.glob('Car_Logo_Dataset/**/*.png', recursive=True)
X, y = load_images(image_paths)

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ANN and GMM models
ann = ANNClassifier(input_shape=X_train.shape[1:], num_classes=y_train.shape[1])
gmm = GMMClassifier(n_components=10)  # Assuming we want to cluster into 10 different car logos

# Compile the ANN model with a custom optimizer
ann.compile(optimizer=tf.keras.optimizers.Adam())

# Create the ensemble model
ensemble = Ensemble(ann, gmm)

# Fit the ensemble model
ensemble.fit(X_train, y_train)

# Predict with the ensemble model
y_pred = ensemble.predict(X_test)

# Since we cannot use built-in accuracy functions, we will use our custom accuracy function
test_accuracy = custom_accuracy(y_test, y_pred)

# Print the custom accuracy
print('Test accuracy (custom metric):', test_accuracy.numpy())


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (544, 32, 32) + inhomogeneous part.